In [201]:
#files to open: train and test data,

from google.colab import files

uploaded = files.upload()


Saving train_y0_rand_v8.csv to train_y0_rand_v8.csv


# Imports and functions

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import class_weight
from sklearn.metrics import recall_score, precision_score, confusion_matrix

In [5]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation

Using TensorFlow backend.


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_loss(epochs, loss, val_loss):
    plt.figure(figsize=(10, 7))
    plt.plot(np.arange(epochs), loss, label='loss')
    plt.plot(np.arange(epochs), val_loss, label='val_loss')
    plt.legend()
    plt.grid(True)
def acc_graph(epochs, train_acc, val_acc):
    x_axis = np.linspace(1, epochs, num=epochs)
    
    plt.figure(figsize=(10, 7))
    plt.plot(x_axis, train_acc, color='g', lw=3, alpha=0.7, label='Train Accuracy')
    plt.plot(x_axis, val_acc, color='orange', lw=3, alpha=0.7, label='Val Accuracy')
    plt.title('Accuracy graph')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

In [0]:
def calculation_classes(y, y_test = None):
    print ('\t\tTRAIN DATA\tTEST DATA')
    print ('Negative\t', np.sum(y==0),'  ',
           round(100*np.sum(y==0)/y.shape[0],2),'%',
           '\t',np.sum(y_test==0),'  ',
           round(100*np.sum(y_test==0)/y_test.shape[0],2),'%'
           '\nPositive\t ', np.sum(y==1),' ',
           round(100*np.sum(y==1)/y.shape[0],2),'%'
           '\t',np.sum(y_test==1),' ',
           round(100*np.sum(y_test==1)/y_test.shape[0],2),'%'
      )

In [0]:
#from utils import model

In [0]:
def scale_0_1_for_samples(X):
  mm = MinMaxScaler()

  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
  
    line = line.reshape(-1, 1)
    X[i] = mm.fit_transform(line).reshape(1,-1)
  return X
 
def scale_standard_for_1sample(X):
  ss = StandardScaler()

  for i in range(X.shape[0]):
    line = X[i]
    X[i] = ss.fit_transform(line.reshape(-1,1)).reshape(1,-1)
  return X

def scale_substraction_mean_for_1sample(X):
  for i in range(X.shape[0]):
    line = X[i]
    #print(line)
    mean = np.mean(line)
    #print(mean)
    X[i] = line - mean
    #print(X[i])
    
  return X
    

In [0]:
def get_mean_for_features(X):
  m = X.shape[0]
  X_mean = np.zeros((m,1))
  for i in range(m):
    X_mean[i] = np.mean(X[i,:], axis=0)
  return X_mean

def scale_standard_for_samples(X, X_mean):
  ss = StandardScaler()
  ss.fit(X_mean)

  X = ss.transform(X)
  return X

In [0]:
def make_predictions(X, y, model, X_ini,out=False,):
  preds = model.predict(X)
  y_predict = np.where(preds > 0.5, 1, 0)
  recall = recall_score(y, y_predict)
  print('Recall = ', recall, 'Precision = ', precision_score(y, y_predict))
  cm  = confusion_matrix(y, y_predict)
  print(cm)
  if out is True:
    for i in range(X.shape[0]):
      if abs(y_predict[i] - y[i])!= 0:
        print(y[i], preds[i], X_ini[i], np.ndarray.tolist(X[i]))
    

# Import data sets

In [0]:
#data_train = pd.read_csv('sources/train.csv')
#data_test = pd.read_csv('sources/test_v7_v8.csv')

data1_train = pd.read_csv('train_y1_v8.csv')
data0_train = pd.read_csv('train_y0_v8.csv')
data0_train_rand = pd.read_csv('train_y0_rand_v8.csv')


X = np.vstack((data1_train.drop(['Y'], axis='columns').values, data0_train.drop(['Y'], axis='columns').values))
X = np.vstack((X, data0_train_rand.drop(['Y'], axis='columns').values))


y = np.hstack((data1_train['Y'].values, data0_train['Y'].values))
y = np.hstack((y, data0_train_rand['Y'].values))

X_ini = X.copy()

In [0]:
data_add = pd.read_csv('train_0_change_1_to_close.csv')

X_add = data_add.drop(['Y'], axis='columns').values
y_add = data_add['Y'].values

X = np.vstack((X, X_add))
y = np.hstack((y, y_add))

In [0]:
data_v4 = pd.read_csv('train_0_v4.csv')

X_add = data_add.drop(['Y'], axis='columns').values
y_add = data_add['Y'].values

X = np.vstack((X, X_add))
y = np.hstack((y, y_add))

In [0]:
scale_X = scale_substraction_mean_for_1sample

In [211]:
X = scale_X(X)

X_train,X_dev,y_train,y_dev = train_test_split(X, y, test_size=0.2, \
                                               shuffle=True, stratify=y)

class_weights_train = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y), y)
print(class_weights)
print(class_weights[0]/(class_weights[1]+class_weights[0]))
calculation_classes(y, y_dev)

X = np.expand_dims(X, axis=2)
X_train = np.expand_dims(X_train, axis=2)
X_dev = np.expand_dims(X_dev, axis=2)

[0.55257412 5.25519126]
0.0951440158053447
		TRAIN DATA	TEST DATA
Negative	 69616    90.49 % 	 13924    90.49 %
Positive	  7320   9.51 %	 1464   9.51 %


In [0]:
data_v4_test = pd.read_csv('test_v4_7615.csv')
X_v4 = data_v4_test.drop(['Y'], axis='columns').values
y_v4 = data_v4_test['Y'].values
test_v4 =  data_v4_test.drop(['Y'], axis='columns').values

X_v4 = scale_X(X_v4)
X_v4 = np.expand_dims(X_v4, axis=2)

In [0]:
data_v7v8 = pd.read_csv('test_v7_v8.csv')

X_v7v8 = data_v7v8.drop(['Y'], axis='columns').values
test_v7v8 =  data_v7v8.drop(['Y'], axis='columns').values
y_v7v8 = data_v7v8['Y'].values

X_v7v8 = scale_X(X_v7v8)
X_v7v8 = np.expand_dims(X_v7v8, axis=2)

In [0]:
#data_test = pd.read_csv('sources/test_v3.csv')
data_v3 = pd.read_csv('test_v3.csv')

X_v3 = data_v3.drop(['Y'], axis='columns').values
test_v3 =  data_v3.drop(['Y'], axis='columns').values
y_v3 = data_v3['Y'].values

X_v3 = scale_X(X_v3)
X_v3 = np.expand_dims(X_v3, axis=2)

# NN on train/dev sets

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


def model_GRU(input_shape, Ty, hidden_size):
    i = Input(shape=input_shape, dtype='float32')
    X = GRU(hidden_size, return_sequences=False)(i)
    
    X = Dense(Ty, activation='sigmoid')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model
  
def model_LSTM(hidden_size):  
  model = Sequential()
  model.add(LSTM(hidden_size, return_sequences=False, input_shape=(Tx,1)))
  model.add(Dense(units=1, activation='sigmoid'))
  
  return model


In [165]:
# For ready model
'''GRU_model = model_GRU((Tx,1),Ty)#LSTM_model.summary()
GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


GRU_model.load_weights('GRU_sequence_weights.h5')'''

"GRU_model = model_GRU((Tx,1),Ty)#LSTM_model.summary()\nGRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\n\n\nGRU_model.load_weights('GRU_sequence_weights.h5')"

In [0]:
Tx=  6
Ty = 1
epochs = 5
batch_size = 32

In [212]:
GRU_model = model_LSTM(16)
opt = Adam(lr=0.02, beta_1=0.9, beta_2=0.999)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
logs = GRU_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train, \
                    initial_epoch=0)

Train on 61548 samples, validate on 15388 samples
Epoch 1/5
 - 12s - loss: 0.2226 - acc: 0.9319 - val_loss: 0.0124 - val_acc: 0.9972
Epoch 2/5
 - 8s - loss: 0.0119 - acc: 0.9965 - val_loss: 0.0055 - val_acc: 0.9985
Epoch 3/5
 - 8s - loss: 0.0061 - acc: 0.9982 - val_loss: 0.0019 - val_acc: 0.9992
Epoch 4/5
 - 8s - loss: 0.0055 - acc: 0.9986 - val_loss: 0.0063 - val_acc: 0.9982
Epoch 5/5
 - 8s - loss: 0.0064 - acc: 0.9982 - val_loss: 0.0335 - val_acc: 0.9903


In [0]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.0005)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [219]:
n_epochs = 5
logs = GRU_model.fit(X_train, y_train , epochs=n_epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train)

Train on 61548 samples, validate on 15388 samples
Epoch 1/5
 - 12s - loss: 9.0021e-04 - acc: 0.9998 - val_loss: 6.2915e-04 - val_acc: 0.9999
Epoch 2/5
 - 8s - loss: 5.4814e-04 - acc: 0.9999 - val_loss: 3.2927e-04 - val_acc: 0.9999
Epoch 3/5
 - 8s - loss: 2.7902e-04 - acc: 0.9999 - val_loss: 1.9602e-04 - val_acc: 0.9999
Epoch 4/5
 - 8s - loss: 2.5352e-04 - acc: 0.9999 - val_loss: 1.5884e-04 - val_acc: 0.9999
Epoch 5/5
 - 8s - loss: 2.0089e-04 - acc: 1.0000 - val_loss: 1.2681e-04 - val_acc: 0.9999


In [220]:
make_predictions(X,y, GRU_model,X_ini, out=False)

Recall =  1.0 Precision =  0.9997268505872713
[[69614     2]
 [    0  7320]]


In [0]:
#make_predictions(X_dev,y_dev, out=True, model=GRU_model)

In [222]:
make_predictions(X_v7v8, y_v7v8, out=True, model=GRU_model, X_ini=test_v7v8)

Recall =  1.0 Precision =  1.0
[[13  0]
 [ 0 46]]


In [223]:
make_predictions(X_v3, y_v3, out=True, model=GRU_model, X_ini=test_v3)

Recall =  0.9622641509433962 Precision =  1.0
[[42  0]
 [ 2 51]]
1.0 [0.03236437] [-0.365 -0.31  -0.262 -0.194 -0.126 -0.056] [[-0.14616666666666664], [-0.09116666666666665], [-0.04316666666666666], [0.024833333333333374], [0.09283333333333335], [0.16283333333333336]]
1.0 [0.12937868] [0.315 0.417 0.526 0.642 0.764 0.9  ] [[-0.27899999999999997], [-0.17699999999999994], [-0.06799999999999995], [0.04800000000000004], [0.17000000000000004], [0.30600000000000005]]


In [224]:
make_predictions(X_v4, y_v4, out=True, model=GRU_model, X_ini=test_v4)

Recall =  0.0 Precision =  0.0
[[71]]


In [0]:
#make_predictions(X_add, y_add, out=True, model=GRU_model, X_ini=X_ini_add)

In [0]:
#plot_loss(epochs, logplot_loss(epos.history.get('loss'), logs.history.get('val_loss'))

In [0]:
#acc_graph(epochs, logs.history.get('acc'), logs.history.get('val_acc'))

In [0]:
GRU_model.save_weights('LSTM_sequence_weights.h5')

In [0]:
files.download('LSTM_sequence_weights.h5')

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
logs = LSTM_model.fit(X_train, y_train , epochs=epochs, batch_size=batch_size, shuffle=True,\
               validation_data=(X_dev, y_dev), verbose=0, class_weight=class_weights_train)

# Final fit NN on all train set

In [0]:
Tx=  6
Ty = 1
epochs = 250
batch_size = 32

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
LSTM_model.fit(X, y , epochs=epochs, batch_size=batch_size, shuffle=True,\
               verbose=0, class_weight=class_weights)
LSTM_model.save_weights('sources/LSTM_sequence_weights.h5')

In [0]:
print(LSTM_model.summary())
test_preds = LSTM_model.predict(X_test)

In [0]:
for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)

In [0]:
LSTM_model = model((Tx,1),Ty)
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('LSTM_sequence_weights.h5')

In [0]:
test_preds = LSTM_model.predict(X_test)

for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)